In [95]:
import pandas as pd

In [127]:
df = pd.read_csv('./data/GPU.csv')

In [3]:
# Prices are in Indian Rupee and in String format, change it to USD and int using API
import requests


class CurrencyExchanger:
    rates = {}

    def __init__(self, url, headers, params):
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        self.rates = data['rates']

    def converter(self, params, amount):
        exchange_rate = self.rates[params['symbols']]
        return round(exchange_rate * amount, 2)

In [4]:
# Instancing the class object

url = 'https://api.apilayer.com/fixer/latest'
headers = {
    'apikey': 'i0khQ80cJeZhfSlcUVClOsXNtOOize60'
}
# We want to convert Indian Rupee to USA Dollar
params = {
    'base': 'INR',
    'symbols': 'USD'
}

currency_exchanger = CurrencyExchanger(url, headers, params)

#Store only the rate and use it onward to data conversion (Note: we only have 1000 request/month)
convert_rate = currency_exchanger.rates[params['symbols']]

In [97]:
# The smallest price is 1000 INR with a comma separator
min(df['MRP'])

'₹1,000'

In [128]:
# Making the price integer
df['MRP'] = df['MRP'].str.replace('₹', '')
df['MRP'] = df['MRP'].str.replace(',', '')
df['MRP'] = df['MRP'].astype(int)

In [129]:
# Price conversion to USD
df['MRP'] = round(df['MRP'] * convert_rate, 2)

In [130]:
df

,Unnamed: 0,GPU,MRP
0,0,ITSNAVAKER NVIDIA CONSISTENT GRAPHICS CARD G21...,28.79
1,1,GMP NVIDIA INKJET PVC ID CARD FOR EPSON PRINTE...,12.02
2,2,ASUS NVIDIA GeForce GT 730 2 GB GDDR5 Graphics...,52.88
3,3,ZEBRONICS NVIDIA ZEB-GT610 2 GB DDR3 Graphics ...,24.63
4,4,ASUS NVIDIA Geforce GT 710 2 GB DDR3 Graphics ...,54.78
...,...,...,...
595,595,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,39.06
596,596,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,18.57
597,597,LipiWorld NVIDIA Pcie Splitter 1 to 4 PCI Rise...,22.59
598,598,Electronicspices AMD Radeon Zotac NTK FD5010U1...,5.84


In [131]:
#Preparing sql export
df.rename(columns={'Unnamed: 0': 'id', 'GPU': 'name', 'MRP': 'mrp_usd'}, inplace=True)

In [132]:
df

,id,name,mrp_usd
0,0,ITSNAVAKER NVIDIA CONSISTENT GRAPHICS CARD G21...,28.79
1,1,GMP NVIDIA INKJET PVC ID CARD FOR EPSON PRINTE...,12.02
2,2,ASUS NVIDIA GeForce GT 730 2 GB GDDR5 Graphics...,52.88
3,3,ZEBRONICS NVIDIA ZEB-GT610 2 GB DDR3 Graphics ...,24.63
4,4,ASUS NVIDIA Geforce GT 710 2 GB DDR3 Graphics ...,54.78
...,...,...,...
595,595,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,39.06
596,596,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,18.57
597,597,LipiWorld NVIDIA Pcie Splitter 1 to 4 PCI Rise...,22.59
598,598,Electronicspices AMD Radeon Zotac NTK FD5010U1...,5.84


In [134]:
# Data validation 
df = df[df['name'].str.contains('Graphics Card')] 
df = df.drop_duplicates(subset=['name', 'mrp_usd'])

In [135]:
# It's kind of redundant data storage, but can make my life easier (and it's a small dataset)
df['architecture'] = df['name'].apply(lambda x: 'NVIDIA' if 'NVIDIA' in x else 'AMD')
df['brand'] = df['name'].str.split(' ').str[0]

In [136]:
df

,id,name,mrp_usd,architecture,brand
0,0,ITSNAVAKER NVIDIA CONSISTENT GRAPHICS CARD G21...,28.79,NVIDIA,ITSNAVAKER
1,1,GMP NVIDIA INKJET PVC ID CARD FOR EPSON PRINTE...,12.02,NVIDIA,GMP
2,2,ASUS NVIDIA GeForce GT 730 2 GB GDDR5 Graphics...,52.88,NVIDIA,ASUS
3,3,ZEBRONICS NVIDIA ZEB-GT610 2 GB DDR3 Graphics ...,24.63,NVIDIA,ZEBRONICS
4,4,ASUS NVIDIA Geforce GT 710 2 GB DDR3 Graphics ...,54.78,NVIDIA,ASUS
...,...,...,...,...,...
595,595,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,39.06,NVIDIA,Nextar
596,596,Nextar NVIDIA VER 009S Riser Cable Black PCIE ...,18.57,NVIDIA,Nextar
597,597,LipiWorld NVIDIA Pcie Splitter 1 to 4 PCI Rise...,22.59,NVIDIA,LipiWorld
598,598,Electronicspices AMD Radeon Zotac NTK FD5010U1...,5.84,AMD,Electronicspices


In [137]:
#sql export
from sqlalchemy import create_engine

#Placeholder localhost login
mysql_config = {
    'user': 'pycharm',
    'password': 'testtest-123',
    'host': 'localhost',
    'database': 'projectcpu'
}

engine = create_engine(
    f"mysql+mysqlconnector://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}/{mysql_config['database']}")

#Use only third of the data atm
df.to_sql(name='gpu', con=engine, if_exists='append', index=False)



191

In [138]:
#quick test query
from sqlalchemy import text
connection = engine.connect()

query = text("select * from gpu")
result = connection.execute(query)

counter = 0
for row in result:
    counter+=1
    print(row)
    
print(counter)
result.close()
connection.close()

(0, 'ITSNAVAKER NVIDIA CONSISTENT GRAPHICS CARD G210 1 GB DDR3 Graphics Card', 28.79, 'NVIDIA', 'ITSNAVAKER')
(1, 'GMP NVIDIA INKJET PVC ID CARD FOR EPSON PRINTER 800 MICRON 230 PCS 1 GB DDR2 Graphics Card', 12.02, 'NVIDIA', 'GMP')
(2, 'ASUS NVIDIA GeForce GT 730 2 GB GDDR5 Graphics Card', 52.88, 'NVIDIA', 'ASUS')
(3, 'ZEBRONICS NVIDIA ZEB-GT610 2 GB DDR3 Graphics Card', 24.63, 'NVIDIA', 'ZEBRONICS')
(4, 'ASUS NVIDIA Geforce GT 710 2 GB DDR3 Graphics Card', 54.78, 'NVIDIA', 'ASUS')
(5, 'GIGABYTE NVIDIA GV-N710D3-2GL 2 GB DDR3 Graphics Card', 48.08, 'NVIDIA', 'GIGABYTE')
(6, 'GEONIX NVIDIA GX GT730 4GB D3 4 GB DDR3 Graphics Card', 35.95, 'NVIDIA', 'GEONIX')
(7, 'GALAX NVIDIA GEFORCE GT 730 4GB DDR3 4 GB GDDR3 Graphics Card', 82.59, 'NVIDIA', 'GALAX')
(8, 'ZOTAC NVIDIA GTX1650AMP 4 GB GDDR6 Graphics Card', 167.71, 'NVIDIA', 'ZOTAC')
(9, 'ASUS AMD Radeon PH-RX550-4G-EVO 4 GB GDDR5 Graphics Card', 120.21, 'AMD', 'ASUS')
(10, 'GEONIX NVIDIA Geonix610 2 GB DDR3 Graphics Card', 22.72, 'NVIDIA